In [1]:
import os

In [2]:
os.chdir("E:/My_Prj/Project/text-summarize")

In [3]:
%pwd

'E:\\My_Prj\\Project\\text-summarize'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from textSummarize.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from textSummarize.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = Path(CONFIG_FILE_PATH),
            params_filepath = Path(PARAMS_FILE_PATH)
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    

In [7]:
import os
import urllib.request as request
import zipfile
from textSummarize.logging import logger
from textSummarize.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            file_name, header = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded file: {file_name} with headers: {header}")
        else:
            logger.info(f"File already exists. Size: {get_size(self.config.local_data_file)} KB")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files to: {unzip_path}")

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-22 15:51:08,144: INFO: common: YAML file config\config.yaml read successfully.]
[2025-08-22 15:51:08,144: INFO: common: YAML file params.yaml read successfully.]
[2025-08-22 15:51:08,144: INFO: common: Directory artifacts created successfully or already exists.]
[2025-08-22 15:51:08,144: INFO: common: Directory artifacts/data_ingestion created successfully or already exists.]
[2025-08-22 15:51:10,188: INFO: 2701001488: Downloaded file: artifacts/data_ingestion/data.zip with headers: Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C963:37B5C3:1D9C2:21775:68A82F7A
Accept-Ranges: bytes
Date: Fri, 22 Aug 2025 08:51:09 GMT
Via: 1.1 v